In [234]:
import os
from io import BytesIO

import pandas
from dotenv import load_dotenv
from minio import Minio

SCORING = {
    'td': 6,
    'rushingyards': 0.1,
    'receivingyards': 0.1,
    'passingyards': 0.05,
    'receptions': 1
}

In [235]:
load_dotenv(override=True)

# Create the Client for MinIO
minio_client = Minio(endpoint=os.getenv('MINIO_ENDPOINT'), access_key=os.getenv('MINIO_KEY'),
                     secret_key=os.getenv('MINIO_SECRET'), cert_check=False, secure=False)

In [236]:
player_files = minio_client.list_objects(bucket_name='football-warehouse', recursive=True,
                                         prefix='players/2023/regular/')

In [237]:
frames = []

for file in player_files:
    response = minio_client.get_object('football-warehouse', file.object_name)
    try:
        content = BytesIO(response.read())
        temp_frame = pandas.read_parquet(content, engine='pyarrow')
        frames.append(temp_frame)
    finally:
        response.close()
        response.release_conn()

stats_frame = pandas.concat(frames, ignore_index=True)


In [238]:
stat_keys = ['receptions', 'receivingyards', 'td', 'passingyards', 'rushingyards']


In [239]:
for key in stat_keys:
    if key == 'td':
        stats_frame.loc[stats_frame['statistic_code'] == 'td', 'points'] = stats_frame[
                                                                               'statistic_value'] * SCORING.get(
            'td', 0)
        continue

    stats_frame.loc[stats_frame['statistic_name'] == key, 'points'] = stats_frame[
                                                                          'statistic_value'] * SCORING.get(
        key, 0)
    

In [240]:
stats_frame.fillna(0, inplace=True)

In [241]:
from plotly import express as px

PLAYER_NAME = 'Patrick Mahomes'

player_stats = stats_frame[stats_frame['player_name'] == PLAYER_NAME]

fig = px.line(player_stats, x="week", y="points", title=f"{PLAYER_NAME} Points per Week")
fig.show()
